In [ ]:
%matplotlib inline
from matplotlib import pylab as plt
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib as mpl
import numpy as np

In [ ]:
from geonotebook.wrappers import RasterDataCollection
import os

In [ ]:
DATA_DIR="/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff/NBAR/"

In [ ]:
def sort_NBAR(a, b):
  am, ay = int(a.split(".")[2][-2:]), int(a.split(".")[3])
  bm, by = int(b.split(".")[2][-2:]), int(b.split(".")[3])

  if ay < by:
    return -1
  elif ay > by:
    return 1
  elif by == ay:
    if am < bm:
      return -1
    elif am > bm:
      return 1
    else:
      return 0
  

PATHS = [DATA_DIR + p for p in sorted(os.listdir(DATA_DIR), sort_NBAR)]

In [ ]:
PATHS

In [ ]:
rdc = RasterDataCollection(PATHS)

In [ ]:
M.add_layer(rdc[:,[1,2,3]], 'NBAR', opacity=0.8)

In [ ]:
M.layers["NBAR"].next()

In [ ]:
M.layers["NBAR"].prev()

In [ ]:
M.layers["NBAR"].seek(0)

In [ ]:
len(M.layers["NBAR"].data)

In [ ]:
def render_timeseries(idx=0):
  M.layers["NBAR"].seek(idx)
  
interact(render_timeseries, idx=(0, len(M.layers["NBAR"].data) - 1))

In [ ]:
M.layers["NBAR"].seek(1)

In [ ]:
M.layers['NBAR'].region.shape

In [ ]:
M.layers['NBAR'].region

In [ ]:
M.remove_layer(M.layers[-1].name)

In [ ]:
M.add_layer(rdc[:,[4]], 'NBAR_NDVI', opacity=0.8)

In [ ]:
M.remove_layer("NBAR_NDVI")

In [ ]:
def ndvi_colormap(numcolors=11, name='custom_div_cmap', 
                  mincol='blue', midcol='beige', maxcol='green'):
  return mpl.colors.LinearSegmentedColormap.from_list(
    name=name, colors=[mincol, midcol, maxcol], N=numcolors)

cmap = ndvi_colormap()


In [ ]:
colormap = [{"color": mpl.colors.rgb2hex(cmap(i)), "quantity": v }
 for i,v in zip(range(cmap.N),np.linspace(min(rdc[:,4].min), max(rdc[:,4].max), cmap.N))]

In [ ]:
colormap

In [ ]:
M.add_layer(rdc[:,4], 'NBAR_NDVI', colormap=colormap, opacity=0.8)

In [ ]:
M.layers['NBAR_NDVI'].next()

In [ ]:
import seaborn as sns
t, lat, lon = M.layers['NBAR_NDVI'].region.shape
X = np.ma.mean(M.layers['NBAR_NDVI'].region.reshape(t, lat * lon), axis=1)

labels = ["{}/{}".format(n.split(".")[2][5:], n.split(".")[3]) for n in 
                         M.layers['NBAR_NDVI'].data.get_names()]

fig, ax = plt.subplots(1,1, figsize=(12, 8))
plt.title("Mean NDVI over selected region")
plt.xticks(range(len(X))[::4], labels[::4])
ax.plot(X)

